In [4]:
from shapely.geometry import shape, Polygon, MultiPolygon
from pymongo import MongoClient

def connect_to_mongodb(database_name, collection_name):
    """
    Connects to MongoDB and returns the MongoDB client and collection.
    """
    client = MongoClient('localhost', 27017)
    db = client[database_name]
    collection = db[collection_name]
    return client, collection

def retrieve_data_from_mongodb(collection):
    """
    Retrieves GeoJSON data from MongoDB collection.
    """
    cursor = collection.find()
    geojson_data = {"type": "FeatureCollection", "features": []}

    for document in cursor:
        geojson_data["features"].append(document)

    return geojson_data

def coordinates_to_polygon(coordinates):
    """
    Converts coordinates to a Shapely Polygon.
    """
    return Polygon(coordinates)

def find_postal_code_for_polygon(target_polygon, postal_code_polygons):
    """
    Finds the postal code for a given polygon among a collection of postal code polygons.
    """
    for postal_code, polygon in postal_code_polygons.items():
        if polygon.intersects(target_polygon):
            return postal_code
    return None

# Example usage
if __name__ == "__main__":
    # MongoDB configuration
    database_name = "ZIP_Poly_Ger_Test"
    collection_name = "ZIP_Poly_Ger_Collection_Test"

    # Connect to MongoDB
    mongo_client, mongo_collection = connect_to_mongodb(database_name, collection_name)

    # Retrieve GeoJSON data from MongoDB
    geojson_data_from_mongo = retrieve_data_from_mongodb(mongo_collection)

    # Load GeoJSON data from MongoDB for comparison
    postal_code_polygons_mongo = {}
    for document in geojson_data_from_mongo['features']:
        postal_code = document.get('postal_code')
        geometry = document.get('geometry', {}).get('coordinates')

        if postal_code and geometry:
            if document['geometry']['type'] == 'Polygon':
                polygon = Polygon(geometry[0])
            elif document['geometry']['type'] == 'MultiPolygon':
                polygon = MultiPolygon([Polygon(shell) for shell in geometry])
            else:
                continue  # Skip unsupported geometry types

            postal_code_polygons_mongo[postal_code] = polygon

    # Example polygon provided
    example_polygon = Polygon([
        (9.4757436, 51.3207645),
        (9.4776032, 51.3206073),
        # ... (add the remaining coordinates)
        (9.4757436, 51.3207645)
    ])

    # Find the postal code for the example polygon
    found_postal_code = find_postal_code_for_polygon(example_polygon, postal_code_polygons_mongo)

    # Output the result
    if found_postal_code:
        print(f"The postal code for the given polygon is: {found_postal_code}")
    else:
        print("No postal code found for the given polygon.")

    # Close MongoDB connection when done
    mongo_client.close()


The postal code for the given polygon is: 34117
